In [1]:
## making sure env variables are set
%env HF_HOME=/hpcwork/ba214121/.cache/huggingface
%env TRANFORMERS_CACHE=/hpcwork/ba214121/.cache/huggingface
%env HF_DATASETS_CACHE=/hpcwork/ba214121/.cache/huggingface
%env WANDB_DIR=/hpcwork/ba214121/.cache/

!echo $HF_DATASETS_CACHE

env: HF_HOME=/hpcwork/ba214121/.cache/huggingface
env: TRANFORMERS_CACHE=/hpcwork/ba214121/.cache/huggingface
env: HF_DATASETS_CACHE=/hpcwork/ba214121/.cache/huggingface
env: WANDB_DIR=/hpcwork/ba214121/.cache/
/hpcwork/ba214121/.cache/huggingface


In [2]:
# You only need to run this once per machine
#!pip install -q -U bitsandbytes
#!pip install -q -U git+https://github.com/huggingface/transformers.git
#!pip install -q -U git+https://github.com/huggingface/peft.git
#!pip install -q -U git+https://github.com/huggingface/accelerate.git
#!pip install -q -U datasets scipy ipywidgets matplotlib
#!pip install -q -U sentencepiece
#!pip install -q -U mistral-common
#!pip install -q -U protobuf
#!pip install -q -U wandb

In [3]:
import pandas as pd
import wandb, os
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ruggsea (ruggero). Use `wandb login --relogin` to force relogin


True

In [4]:
from datasets import load_dataset

# load ruggsea/stanford-encyclopedia-of-philosophy_chat_multi_turn and then split it into train and eval

dataset = load_dataset("ruggsea/stanford-encyclopedia-of-philosophy_chat_multi_turn")

train = dataset['train']

# 1% of the data is used for eval, 99% for training
eval_train = train.train_test_split(test_size=0.01)
eval = eval_train['test']
train = eval_train['train']


# create a folder to save the data

os.makedirs('chat_finetuning', exist_ok=True)

# save the data in jsonlines format

train.to_json('chat_finetuning/train.jsonl', orient='records', lines=True)
eval.to_json('chat_finetuning/eval.jsonl', orient='records', lines=True)

Creating json from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

847047

In [5]:
# print the number of examples in the training and eval sets
print(f"Number of training examples: {len(train)}")
print(f"Number of eval examples: {len(eval)}")

Number of training examples: 11784
Number of eval examples: 120


In [6]:
# from accelerate import FullyShardedDataParallelPlugin, Accelerator
# from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

# fsdp_plugin = FullyShardedDataParallelPlugin(
#     state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
#     optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
# )

# accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"


bnb_config = BitsAndBytesConfig(
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=torch.bfloat16
)



model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

In [9]:
max_length=4096 # this value is tentative

train_chats=tokenizer.apply_chat_template(train["conversation"],
        truncation=True,
        max_length=max_length,
        padding="max_length",
)

eval_chats=tokenizer.apply_chat_template(eval["conversation"],
        truncation=True,
        max_length=max_length,
        padding="max_length",
)

In [10]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [11]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [12]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 88121344 || all params: 4628721664 || trainable%: 1.9037944036550305


In [13]:
if torch.cuda.device_count() > 1: # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True

torch.cuda.device_count()

1

In [14]:
# model = accelerator.prepare_model(model)

In [ ]:
import transformers
from datetime import datetime
from transformers import EarlyStoppingCallback


# setting up wandb things
project = "stanford-philosophy-chat_finetune"
base_model_name = "llama3.1-8B-Instruct"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name
os.environ["WANDB_PROJECT"]=project
os.environ["WANDB_LOG_MODEL"] = "checkpoint"


model.resize_token_embeddings(len(tokenizer))

trainer = transformers.Trainer(
    model=model,
    train_dataset=train_chats,
    eval_dataset=eval_chats,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=0,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        gradient_checkpointing=True,
        num_train_epochs=1,
        #max_steps=500,
        learning_rate=2e-5, # Want a small lr for finetuning
        bf16=True,
        optim="paged_adamw_8bit",
        logging_steps=15,              # When to start reporting loss
        logging_dir="./logs",        # Directory for storing logs
        save_strategy="steps",       # Save the model checkpoint every logging step
        save_steps=3000,                # Save checkpoints every n steps
        eval_strategy="steps", # Evaluate the model every logging step
        eval_steps=500,               # Evaluate and save checkpoints every n steps
        do_eval=True,                # Perform evaluation at the end of training
        report_to="wandb",           # Comment this out if you don't want to use weights & baises
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"          # Name of the W&B run (optional)
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


/cvmfs/jupyter.hpc.rwth.de/clients/pytorch231-c23/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
500,1.343300,1.282616
1000,1.224500,1.257805
1500,1.200700,1.243441
2000,1.278700,1.239794
2500,1.223600,1.234607
3000,1.225900,1.231025
3500,1.137900,1.249657
4000,1.279600,1.295451
4500,1.429000,1.375770
5000,1.214100,1.458103


/home/ba214121/.local/lib/python3.11/site-packages/peft/utils/save_and_load.py:203: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
wandb: Adding directory to artifact (./llama3.1-8B-Instruct-stanford-philosophy-chat_finetune/checkpoint-3000)... Done. 26.8s
/cvmfs/jupyter.hpc.rwth.de/clients/pytorch231-c23/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/ba214121/.local/lib/python3.11/site-packages/peft/utils/save_and_load.py:203

In [7]:
import gc
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


from peft import PeftModel

# Flush memory
# del trainer, model
gc.collect()
torch.cuda.empty_cache()

base_model_name ="meta-llama/Meta-Llama-3-8B-Instruct"
new_model_name="ruggsea/Llama3.1-Chat-stanford-encyclopedia-philosophy"


adapter="./llama3.1-8B-Instruct-stanford-philosophy-chat_finetune/checkpoint-11784"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

# Merge adapter with base model
model = PeftModel.from_pretrained(model, adapter)
model = model.merge_and_unload()
model.push_to_hub(new_model_name, use_temp_dir=True)
tokenizer.push_to_hub(new_model_name, use_temp_dir=True)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ruggsea/Llama3.1-Chat-stanford-encyclopedia-philosophy/commit/92abad9292e391616bfc0556a6d43dcbd068bf05', commit_message='Upload tokenizer', commit_description='', oid='92abad9292e391616bfc0556a6d43dcbd068bf05', pr_url=None, pr_revision=None, pr_num=None)

## Testing the model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"



base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # LLama3.1, same as before
    #quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
)

eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "mistral-stanford-philosophy-finetune/checkpoint-1185")

eval_prompt = "Please explain the allegory of the cave to me."
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=256, repetition_penalty=1.15)[0], skip_special_tokens=True))